In [62]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("maso0dahmed/football-players-data")

# print("Path to dataset files:", path)

In [63]:

import pandas as pd
df  = pd.read_csv('/Users/baonguyen/.cache/kagglehub/datasets/maso0dahmed/football-players-data/versions/1/fifa_players.csv')

In [64]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [65]:
# One-hot encode 'positions' with a prefix
position_dummies = df['positions'].str.get_dummies(sep=',').add_prefix('pos_')

# One-hot encode 'national_team_position' with a different prefix
ntp_dummies = df['national_team_position'].str.get_dummies(sep=',').add_prefix('ntp_')

# Concatenate the dummy dataframes to the original dataframe
df = pd.concat([df, position_dummies, ntp_dummies], axis=1)

# Optionally, drop the original columns
df.drop(['positions', 'national_team_position'], axis=1, inplace=True)


In [66]:
from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
le = LabelEncoder()

# Fit and transform the 'preferred_foot' column, creating a new column
df['preferred_foot_encoded'] = le.fit_transform(df['preferred_foot'])
df.drop('preferred_foot',axis=1,inplace=True)


In [67]:
for col in df.columns:
    mode_val = df[col].mode()[0]  # get the first mode value
    df[col].fillna(mode_val, inplace=True)


/var/folders/yw/97cz46nn1hv_4clq2jcjt5j80000gn/T/ipykernel_7936/211245949.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_val, inplace=True)
/var/folders/yw/97cz46nn1hv_4clq2jcjt5j80000gn/T/ipykernel_7936/211245949.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [68]:
# drop uneccessary cols
df.drop(['full_name','birth_date','nationality','value_euro','wage_euro','body_type','release_clause_euro','national_team','national_jersey_number'],axis=1,inplace=True)


In [69]:
df.head()

,name,age,height_cm,weight_kgs,overall_rating,potential,international_reputation(1-5),weak_foot(1-5),skill_moves(1-5),national_rating,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,freekick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,pos_CAM,pos_CB,pos_CDM,pos_CF,pos_CM,pos_GK,pos_LB,pos_LM,pos_LW,pos_LWB,pos_RB,pos_RM,pos_RW,pos_RWB,pos_ST,ntp_CAM,ntp_CB,ntp_CDM,ntp_CF,ntp_CM,ntp_GK,ntp_LAM,ntp_LB,ntp_LCB,ntp_LCM,ntp_LDM,ntp_LF,ntp_LM,ntp_LS,ntp_LW,ntp_LWB,ntp_RAM,ntp_RB,ntp_RCB,ntp_RCM,ntp_RDM,ntp_RES,ntp_RF,ntp_RM,ntp_RS,ntp_RW,ntp_RWB,ntp_ST,ntp_SUB,preferred_foot_encoded
0,L. Messi,31,170.18,72.1,94,94,5,4,4,82.0,86,95,70,92,86,97,93,94,89,96,91,86,93,95,95,85,68,72,66,94,48,22,94,94,75,96,33,28,26,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,C. Eriksen,27,154.94,76.2,88,89,3,5,4,78.0,88,81,52,91,80,84,86,87,89,91,76,73,80,88,81,84,50,92,58,89,46,56,84,91,67,88,59,57,22,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,P. Pogba,25,190.50,83.9,88,91,4,4,5,84.0,80,75,75,86,85,87,85,82,90,90,71,79,76,82,66,90,83,88,87,82,78,64,82,88,82,87,63,67,67,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,L. Insigne,27,162.56,59.0,88,88,3,4,4,83.0,86,77,56,85,74,90,87,77,78,93,94,86,94,83,93,75,53,75,44,84,34,26,83,87,61,83,51,24,22,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,K. Koulibaly,27,187.96,88.9,88,91,3,3,2,75.0,30,22,83,68,14,69,28,28,60,63,70,75,50,82,40,55,81,75,94,15,87,88,24,49,33,80,91,88,87,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [70]:
from sklearn.preprocessing import StandardScaler
# Select numeric columns
# Select all numeric columns first
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Filter out columns that start with 'pos' or 'ntp'
filtered_numeric_cols = [col for col in numeric_cols if not (col.startswith('pos') or col.startswith('ntp') or col.endswith('(1-5)'))]

print(filtered_numeric_cols)

# Scale the numeric data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[filtered_numeric_cols])

# Convert the scaled data back into a DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=[f'scaled_{col}' for col in filtered_numeric_cols])

# Merge the scaled data back into the original DataFrame
df = pd.concat([df, scaled_df], axis=1)

# Now drop the original numeric columns
df.drop(columns=filtered_numeric_cols, inplace=True)

['age', 'height_cm', 'weight_kgs', 'overall_rating', 'potential', 'national_rating', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'freekick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'vision', 'penalties', 'composure', 'marking', 'standing_tackle', 'sliding_tackle', 'preferred_foot_encoded']


In [71]:
df.head()

,name,international_reputation(1-5),weak_foot(1-5),skill_moves(1-5),positioning,pos_CAM,pos_CB,pos_CDM,pos_CF,pos_CM,pos_GK,pos_LB,pos_LM,pos_LW,pos_LWB,pos_RB,pos_RM,pos_RW,pos_RWB,pos_ST,ntp_CAM,ntp_CB,ntp_CDM,ntp_CF,ntp_CM,ntp_GK,ntp_LAM,ntp_LB,ntp_LCB,ntp_LCM,ntp_LDM,ntp_LF,ntp_LM,ntp_LS,ntp_LW,ntp_LWB,ntp_RAM,ntp_RB,ntp_RCB,ntp_RCM,ntp_RDM,ntp_RES,ntp_RF,ntp_RM,ntp_RS,ntp_RW,ntp_RWB,ntp_ST,ntp_SUB,scaled_age,scaled_height_cm,scaled_weight_kgs,scaled_overall_rating,scaled_potential,scaled_national_rating,scaled_crossing,scaled_finishing,scaled_heading_accuracy,scaled_short_passing,scaled_volleys,scaled_dribbling,scaled_curve,scaled_freekick_accuracy,scaled_long_passing,scaled_ball_control,scaled_acceleration,scaled_sprint_speed,scaled_agility,scaled_reactions,scaled_balance,scaled_shot_power,scaled_jumping,scaled_stamina,scaled_strength,scaled_long_shots,scaled_aggression,scaled_interceptions,scaled_vision,scaled_penalties,scaled_composure,scaled_marking,scaled_standing_tackle,scaled_sliding_tackle,scaled_preferred_foot_encoded
0,L. Messi,5,4,4,94,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.154918,-0.339789,-0.451903,3.986456,3.681038,6.400693,1.964115,2.527553,1.018860,2.249510,2.429150,2.185336,2.476186,2.921234,2.356027,2.238551,1.753477,1.439751,2.000640,3.655821,2.196326,1.710590,0.260118,0.557021,0.067101,2.426684,-0.449392,-1.188102,2.867672,1.685137,3.210247,-0.706843,-0.910432,-0.925803,-1.817256
1,C. Eriksen,3,5,4,84,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.304863,-1.426106,0.126908,3.124825,2.865533,2.709377,2.072323,1.814731,-0.008459,2.182228,2.092117,1.504400,2.098545,2.522718,2.356027,1.942259,0.753539,0.556739,1.122649,2.884507,1.208577,1.652956,-1.277636,1.813530,-0.571081,2.169334,-0.564377,0.450131,2.655742,1.179141,2.522088,0.590770,0.427554,-1.113727,0.550280
2,P. Pogba,4,4,5,82,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-0.120165,1.108633,1.213944,3.124825,3.191735,8.246351,1.639488,1.509235,1.304227,1.845817,2.372978,1.661539,2.044596,2.238063,2.420874,1.883000,0.420226,0.964283,0.852498,2.223381,0.150275,1.998761,1.541579,1.562228,1.742329,1.809044,1.275384,0.835597,2.443813,2.127884,2.436068,0.790403,0.888928,1.000417,0.550280
3,L. Insigne,3,4,4,83,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.304863,-0.882947,-2.301275,3.124825,2.702432,7.323522,1.964115,1.611067,0.219834,1.778535,1.755085,1.818678,2.152494,1.953409,1.642720,2.060775,1.953464,1.439751,2.068177,2.333569,2.055219,1.134248,-1.021343,0.745497,-1.687900,1.911984,-1.254288,-0.995369,2.373169,0.799644,2.091988,0.191505,-1.094982,-1.113727,0.550280
4,K. Koulibaly,3,3,2,24,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.304863,0.927580,1.919811,3.124825,3.191735,-0.059110,-1.065730,-1.189305,1.760813,0.634737,-1.615240,0.718703,-1.030483,-0.836207,0.475489,0.283024,0.353563,0.692587,-0.903484,2.223381,-1.684116,-0.018436,1.370718,0.745497,2.300738,-1.639446,1.792817,1.991997,-0.311272,-0.971342,1.833928,2.187833,1.857815,1.940036,0.550280


In [76]:
from sklearn.metrics import pairwise_distances

# Extract only the numeric columns for positions
clean_df = df.drop(['name','scaled_age'], axis=1).values


In [77]:
import numpy as np
import gower
# Calculate the pairwise distances (using Euclidean as an example)
dist_matrix = pairwise_distances(clean_df, metric='euclidean')
# dist_matrix_gower = gower.gower_matrix(clean_df)
def get_top_n_nearest(dist_matrix, index, n=10):
    """
    Returns the indices and distances of the top n nearest neighbors
    for the given index (excluding the index itself).
    """
    # Get the distances for the given index
    distances = dist_matrix[index]
    
    # Sort the indices based on distance
    sorted_indices = np.argsort(distances)
    
    # Remove the index itself (distance = 0)
    sorted_indices = sorted_indices[sorted_indices != index]
    
    # Select the top n indices and their corresponding distances
    top_n_indices = sorted_indices[:n]
    top_n_distances = distances[top_n_indices]
    
    return top_n_indices, top_n_distances



In [78]:
# Example: For player at index 0, get top 10 nearest neighbors
index = 0  # change this to any valid index
nearest_indices, nearest_distances = get_top_n_nearest(dist_matrix, index, n=10)

print(f"Top 10 nearest neighbors for index {index}:")
for i, (idx, dist) in enumerate(zip(nearest_indices, nearest_distances), start=1):
    print(f"{i}. Index: {idx}, Distance: {dist}")
print('\n\n')
print(df.loc[index, 'name'])

for i,idx in enumerate(nearest_indices):
    print(i+1,'.',df.loc[idx, 'name'])


Top 10 nearest neighbors for index 0:
1. Index: 7, Distance: 5.253222479126249
2. Index: 17944, Distance: 5.820497277981386
3. Index: 13, Distance: 5.997051908222823
4. Index: 17938, Distance: 6.408968601316264
5. Index: 17943, Distance: 6.88443296133147
6. Index: 17937, Distance: 6.973807282132689
7. Index: 17905, Distance: 7.330501181528037
8. Index: 9, Distance: 7.4009795135334135
9. Index: 31, Distance: 8.113538385548676
10. Index: 17935, Distance: 8.113998985134321



L. Messi
1 . S. Agüero
2 . Cristiano Ronaldo
3 . A. Griezmann
4 . L. Suárez
5 . Neymar Jr
6 . H. Kane
7 . T. Müller
8 . E. Cavani
9 . M. Icardi
10 . R. Lewandowski


In [54]:
import streamlit as st
# --- Streamlit App ---
st.title("Player Nearest Neighbors Finder")

# Text input to enter a player's name
player_input = st.text_input("Enter player's name:")

if player_input:
    # Filter df for names matching the input (case insensitive)
    suggestions = df[
    (df['full_name'].str.contains(player_input, case=False, na=False)) | 
    (df['name'].str.contains(player_input, case=False, na=False))
]
    
    if not suggestions.empty:
        # Display a selectbox with the suggestions
        selected_player = st.selectbox("Select a player", suggestions['name'].tolist())
        
        # Find the index of the selected player in the DataFrame
        player_index = df[df['name'] == selected_player].index[0]
        
        st.write(f"Selected player: **{selected_player}**")
        
        # Get the top 10 nearest neighbors using the precomputed distance matrix
        nearest_indices, nearest_distances = get_top_n_nearest(dist_matrix, player_index, n=10)
        
        st.subheader("Top 10 Nearest Neighbors:")
        for idx, dist in zip(nearest_indices, nearest_distances):
            st.write(f"- **{df.loc[idx, 'name']}** (Distance: {dist:.2f})")
    else:
        st.write("No player found matching that name.")

2025-02-19 11:38:45.345 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:38:45.375 
  command:

    streamlit run /Users/baonguyen/.pyenv/versions/miniconda3-latest/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-02-19 11:38:45.376 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:38:45.376 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:38:45.376 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:38:45.376 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 11:38:45.376 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19